In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *


spark = SparkSession.builder.appName("taxi") \
.config("spark.sql.pyspark.jvm", "false") \
.config("spark.executor.memory", "8g") \ 
.getOrCreate()


#.master("local[*]") \
#.config("spark.driver.memory", "8g") \
#.withColumn("event_date", expr ("DATE_TRUNC('day', event_time)"))


In [7]:
df_sample = (spark.read
        .format("parquet")
        .load("data/taxi/yellow_taxi/sample/*"))
	
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

# df_sample = (spark.read
#         .format("parquet")
#         .load("data/taxi/yellow_taxi/2011/yellow_tripdata_2011-01.parquet"))
	
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


In [4]:
df = (spark.read
        .format("parquet")
        .load("data/taxi/green_taxi/*"))

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/home/jovyan/data/taxi/green_taxi/*.

In [ ]:
def compare_schemas(df_sample, df):
    if df_sample.schema == df.schema:
        print("Schematy są IDENTYCZNE")
    else:
        print("RÓŻNICE W SCHEMATACH:")
        print("=" * 60)
       
        s1 = {f.name: (str(f.dataType), f.nullable) for f in df_sample.schema.fields}
        s2 = {f.name: (str(f.dataType), f.nullable) for f in df.schema.fields}
       
        all_cols = sorted(set(s1.keys()) | set(s2.keys()))
       
        for col in all_cols:
            in_df_sample = col in s1
            in_df = col in s2
           
            if not in_df_sample:
                print(f" tylko w df → {col:25} {s2[col][0]:20} (nullable={s2[col][1]})")
            elif not in_df:
                print(f" tylko w df_sample → {col:25} {s1[col][0]:20} (nullable={s1[col][1]})")
            elif s1[col] != s2[col]:
                print(f" RÓŻNICA → {col:25} df_sample: {s1[col][0]:15} (null={s1[col][1]}) | "
                      f"df: {s2[col][0]:15} (null={s2[col][1]})")
                
compare_schemas(df_sample, df)

In [ ]:
df = df.withColumnRenamed("lpep_pickup_datetime", "tpep_pickup_datetime") \
.withColumnRenamed("lpep_dropoff_datetime", "tpep_dropoff_datetime") \
.drop("trip_type") 

In [ ]:
df = df \
.withColumn("VendorID", col("VendorID").cast("integer")) \
.withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast("timestamp_ntz")) \
.withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime").cast("timestamp_ntz")) \
.withColumn("passenger_count", col("passenger_count").cast("integer")) \
.withColumn("trip_distance", col("trip_distance").cast("float")) \
.withColumn("RatecodeID", col("RatecodeID").cast("integer")) \
.withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("boolean")) \
.withColumn("PULocationID", col("PULocationID").cast("integer")) \
.withColumn("DOLocationID", col("DOLocationID").cast("integer")) \
.withColumn("payment_type", col("payment_type").cast("integer")) \
.withColumn("fare_amount", col("fare_amount").cast("float")) \
.withColumn("extra", col("extra").cast("float")) \
.withColumn("mta_tax", col("mta_tax").cast("float")) \
.withColumn("tip_amount", col("tip_amount").cast("float")) \
.withColumn("tolls_amount", col("tolls_amount").cast("float")) \
.withColumn("improvement_surcharge", col("improvement_surcharge").cast("float")) \
.withColumn("total_amount", col("total_amount").cast("float")) \
.withColumn("congestion_surcharge", col("congestion_surcharge").cast("float")) 
#.withColumn("airport_fee", col("airport_fee").cast("float"))

In [6]:
df_sample.describe()

summary,VendorID,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
count,803860,803860,803860,803860,803860,803860,803860,803860,803860,803860,803860,803860,0,14,803860,803860,355,0
mean,1.75689796730774,NULL,1.1213818326574279,123.55262981116115,136.5182904983455,1.4742915433035604,2.8166176324234486,11.838116450625694,0.3580091558231532,0.4890422834821984,0.9025804742118398,0.10397971039735683,NULL,0.17142857142857143,13.692027032069825,1.6540056726295624,1.0056338028169014,NULL
stddev,0.42895648181754603,NULL,0.7220787829855728,78.8331687542218,77.36119121329234,1.1952107641060672,2.836507226382903,9.725503037643639,0.7796521137299455,0.07368394856077218,2.0161152769662336,2.29668050527423,NULL,0.15406577730392865,11.095465318837974,0.49742138443190953,0.07495257246871866,NULL
min,1,N,1,1,1,0,0.0,-3.5,0.0,-0.5,0.0,0.0,NULL,0.0,-4.0,1,1.0,NULL
max,2,Y,99,265,265,9,115.61,2924.5,565.84,5.13,380.0,1481.87,NULL,0.3,2925.0,4,2.0,NULL


In [11]:
df_sample.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: boolean (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- airport_fee: float (nullable = true)



In [ ]:
#############ZMIENIĆ ROK!!!############

(df
.write
.mode("overwrite")
.option("parquet.compression", "gzip")
.parquet("data/taxi/green_taxi/formated/"))
##############################^^^^^^#########################

In [ ]:
df.groupBy("passenger_count") \
  .count() \
  .orderBy(col("passenger_count").asc()) \
  .show(20)

In [ ]:
df.orderBy("tpep_pickup_datetime", ascending=False).show()